In [ ]:
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'  # always print last expr.

In [ ]:
from dataclasses import *
from typing import *
from typing_extensions import *

In [ ]:
@dataclass
class Foo:
    x: int
    y: int


class Bar:
    pass


set(dir(Foo)) - set(dir(Bar))

In [ ]:
Foo.__match_args__

In [ ]:
from tsdm.types.protocols import Dataclass

In [ ]:
issubclass(Foo, Dataclass)